In [2]:
import datetime as dt
import matplotlib.dates as mdates
from obspy import UTCDateTime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob

In [84]:
starttimes=[]
endtimes=[]
net=[]
nooftimes=input("howmuch?")
nooftimes=int(nooftimes)
while nooftimes !=0:
    starttime=input("start time?")
    endtime=input("endtime time?")
    network=input("Network?")
    startt=UTCDateTime(starttime)
    endt=UTCDateTime(endtime)
    starttimes.append(starttime)
    endtimes.append(endtime)
    net.append(network)
    nooftimes=nooftimes-1
else:
    pass
print("start times are",starttimes)
print("endtimes are",endtimes)
    

howmuch? 1
start time? 2018-05-01
endtime time? 2018-05-02
Network? TA


start times are ['2018-05-01']
endtimes are ['2018-05-02']


In [76]:
sta=input("Station?")
loc=input("location?")
cha=input("channel?")

Station? L17K
location? *
channel? BHZ


In [85]:
import requests
for z in range (len(starttimes)):
    url='https://service.iris.edu/mustang/noise-psd/1/query?target='+net[z]+"."+sta+"."+loc+"."+cha+".M&starttime="+str(starttimes[z])+"&endtime="+str(endtimes[z])+"&format=xml"
    print(url)
    resp = requests.get(url)
    with open('pdf'+str(z)+".xml", 'w') as foutput:
        foutput.write(resp.content.decode('utf-8'))

https://service.iris.edu/mustang/noise-psd/1/query?target=TA.L17K.*.BHZ.M&starttime=2018-05-01&endtime=2018-05-02&format=xml


In [136]:
files=[]
for name in glob.glob('/home/sjohn/spectrogram/*xml'):
    files.append(name)
    print(name)

/home/sjohn/spectrogram/pdf0.xml
/home/sjohn/spectrogram/pdf1.xml
/home/sjohn/spectrogram/combine.xml


In [88]:
os.system('xmlmerge *.xml > combine.xml')

0

In [4]:
os.cpu_count()

8

In [7]:
frames=[]
name= pd.read_xml("combine.xml", xpath="/PsdRoot/Psds/Psd")
print(len(name))
j=0
for i in range(len(name)):
    if i%2==0:
        print(i)
    stu=name.iloc[i]['target']
    stu=(stu[:2])
    if stu=="AK":
        j+=1
        f= pd.read_xml("combine.xml", xpath="/PsdRoot/Psds[2]/Psd["+str(j)+"]/value[@freq<20]")
        frames.append(f)
    else:
        g= pd.read_xml("combine.xml", xpath="/PsdRoot/Psds[1]/Psd["+str(i+1)+"]/value")
        frames.append(g)
result=pd.concat(frames)

    

71687
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116


KeyboardInterrupt: 

In [ ]:
ma=np.zeros((96,len(name)))
co=0
r=-1
for i in range (0,len(result)):
    if ((r+1)/96) == 1:
        r=0
        co+=1
    else :
        r+=1
    ma[r,co]=result.iloc[i,1]


In [142]:
print(ma)
np.shape(ma)

[[-176.20278846 -176.82448846 -177.53218846 ..., -174.80929336
  -175.11279336 -178.42309336]
 [-176.71583533 -177.33753533 -178.04523533 ..., -177.17574297
  -174.32164297 -175.73834297]
 [-176.91062533 -177.56882533 -176.96092533 ..., -177.6106308
  -174.7565308  -176.1732308 ]
 ..., 
 [-145.25587467 -144.62047467 -144.69067467 ..., -150.26404489
  -149.88324489 -149.44254489]
 [-128.58393356 -127.91563356 -127.99553356 ..., -149.96448589
  -149.56318589 -149.11748589]
 [ -76.7271174   -76.0281174   -76.1072174  ..., -149.23401874
  -148.79931874 -148.31381874]]


(96, 94)

In [ ]:
final=np.delete(ma, range(1, ma.shape[1], 2), axis=1)
print(final)
np.shape(final)

In [ ]:
startt=UTCDateTime(starttimes[0])
print(startt)
endt=UTCDateTime(endtimes[-1])
print(endt)
l=((startt-endt)/8)
print(l)
time=np.arange(startt,endt,-l)
print(time)
np.shape(time)

In [ ]:
time_mpl = [t.matplotlib_date for t in time]
time_mpl=[t.strftime('%b, %d,%y') for t in time]

In [ ]:
date_format=mdates.DateFormatter('%b, %d,%y')

In [ ]:
freq =np.arange(0,20,4)

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 2))
#c = ax1.imshow(final, cmap='plasma')
c = ax1.pcolormesh(final, cmap='plasma')
#fig.set_figwidth(10)
#fig.set_figheight(3)
ax1.set_xticklabels(time_mpl)
ax1.set_yticklabels(freq)
ax1.title.set_text(sta)
fig.savefig("finalspectro1.png")